In [ ]:
import os
import tempfile
import uuid

from training_datasets import sdata_xenium

# The OUTPUT_DIR is the directory where the SpatialData .zarr will be saved. Change it to your output directory of choice.
OUTPUT_DIR =  tempfile.gettempdir()

sdata=sdata_xenium( path = None, output= os.path.join( OUTPUT_DIR, f"sdata_{uuid.uuid4()}.zarr" ) ) # on Windows, set path (e.g. to r"c:\tmp")
sdata

Download an `instanseg` model.

Make sure to install `instanseg`:

`pip install git+https://github.com/instanseg/instanseg.git@main`


In [ ]:
import os

import torch
from instanseg import InstanSeg

# Call the function to download and extract the models
instanseg_brightfield = InstanSeg("brightfield_nuclei", verbosity=1,  device="cpu")
instanseg_fluorescence = InstanSeg("fluorescence_nuclei_and_cells", verbosity=1 ,device="cpu")

# or load the model from a path
path_model = os.path.join( os.environ.get( "INSTANSEG_BIOIMAGEIO_PATH" ), "fluorescence_nuclei_and_cells/0.1.0/instanseg.pt" )
instanseg_fluorescence= torch.load( path_model )
instanseg_fluorescence=InstanSeg( model_type=instanseg_fluorescence, device="cpu" )

Distributed segmentation using `harpy.im.segment`.

In [ ]:
import harpy as hp

crd = [10000,15000,10000,15000]

sdata=hp.im.segment(
    sdata,
    img_layer="morphology_focus_global",
    output_labels_layer=[  "masks_nuclei_iou_harpy", "masks_cells_iou_harpy" ],
    output_shapes_layer=[  "shapes_nuclei_harpy", "shapes_cells_harpy" ],
    labels_layer_align="masks_cells_iou_harpy",
    model=hp.im.instanseg_callable,
    output="all_outputs",
    chunks=(1000,1000),
    depth=300,
    crd = crd,
    device="cpu",
    #instanseg_model = instanseg_fluorescence,
    instanseg_model = path_model, # load it in every worker, because torchscript model not serializable
    overwrite=True,
    pixel_size = 0.2125,
          )

Visualization

In [ ]:
hp.pl.plot_shapes(
    sdata,
    img_layer="morphology_focus_global",
    shapes_layer="shapes_cells_harpy",
    crd = [ 11000,12000,11000,12000 ],
    figsize=( 5,5, ),
    channel="DAPI",
        )